In [1]:
%run TAQplots.ipynb

In [2]:
#OrderLog = 'OrderLog20020812'
OrderLog = 'OrderLog20071001'
OrderLog_df = pd.read_csv('./resources/' + OrderLog + '.txt')

#
#OrderLog_df = OrderLog_df[OrderLog_df.SECCODE == 'LKOH']
#

OrderLog_df = OrderLog_df.astype({"NO":"int",
                                                  "SECCODE":"string",
                                                  "BUYSELL":"string",
                                                  "TIME":"string",
                                                  "ORDERNO":"int",
                                                  "ACTION":"int",
                                                  "PRICE":"float",
                                                  "VOLUME":"int",
                                                  "TRADENO":pd.Int64Dtype(),
                                                  "TRADEPRICE":"float"
                                                 })
OrderLog_df.head()

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
0,1,EESR,B,103000000,1,1,28.0000,10000,<NA>,NaN
1,2,GAZP,S,103000000,2,1,289.0000,20,<NA>,NaN
2,3,RU14TATN3006,S,103000000,3,1,143.6700,1380,<NA>,NaN
3,4,URSI,S,103000000,4,1,1.6200,1100,<NA>,NaN
4,5,TGKI,B,103000000,5,1,0.0068,20000000,<NA>,NaN


In [3]:
class TAQ2TAQplots:
    def __init__(self, taq_df,
              row_id_col, # unique row id
              sec_col, # column with security code
              link_col, # orderno or some mark linking trades and quotes
              trdno_col
             ):
        self.taq_df = taq_df.copy()
        self.row_id_col = row_id_col
        self.sec_col = sec_col
        self.link_col = link_col
        self.trdno_col = trdno_col
        self.secs = pd.unique(taq_df[sec_col])
        self.vertex_count_dic = {k:v for (k, v) in zip(self.secs, [0 for i in range(len(self.secs))])}
        self.rowid2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.lnk2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.trdno2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.edge_tpls = {k:v for (k, v) in zip(self.secs, [[] for i in range(len(self.secs))])}
        self.rows_to_edges()
        self.Gs = {k:v for (k, v) in zip(self.secs, [Graph(self.edge_tpls[sec],
                                                           self.vertex_count_dic[sec]) for sec in self.secs])}
        self.TAQpps = {k:v for (k, v) in zip(self.secs, [TAQplots(self.Gs[sec]) for sec in self.secs])}

    def sec_rowid_lnk2edges(self, rowid, sec, lnk, trdno): 
        # give 0, 1, 2, ... vertex names to all raw ids and "links" (eg ordernos)
        if rowid not in self.rowid2gkey_dic[sec]:
            self.rowid2gkey_dic[sec][rowid] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        if lnk not in self.lnk2gkey_dic[sec]:
            self.lnk2gkey_dic[sec][lnk] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        if trdno not in self.trdno2gkey_dic[sec]:
            self.trdno2gkey_dic[sec][trdno] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        self.edge_tpls[sec].append((self.rowid2gkey_dic[sec][rowid], 
                               self.lnk2gkey_dic[sec][lnk]))
        if pd.notna(trdno):
            self.edge_tpls[sec].append((self.rowid2gkey_dic[sec][rowid], 
                               self.trdno2gkey_dic[sec][trdno]))

    def rows_to_edges(self):
        [self.sec_rowid_lnk2edges(rowid, sec, lnk, trdno) 
         for rowid, sec, lnk, trdno in zip(self.taq_df[self.row_id_col],
                                    self.taq_df[self.sec_col],
                                    self.taq_df[self.link_col],
                                    self.taq_df[self.trdno_col])]
        

In [4]:
t2tpp = TAQ2TAQplots(OrderLog_df, 'NO', 'SECCODE', 'ORDERNO', 'TRADENO')
OrderLog_df['TAQPLOTNO'] = [t2tpp.TAQpps[sec].TAQplotnos[t2tpp.rowid2gkey_dic[sec][rowid]] 
                                    for (sec, rowid) in zip(OrderLog_df['SECCODE'], 
                                                            OrderLog_df['NO'])]
OrderLog_df.to_csv('./resources/' + OrderLog + '.csv')
OrderLog_df

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,TAQPLOTNO
0,1,EESR,B,103000000,1,1,28.0000,10000,<NA>,NaN,0
1,2,GAZP,S,103000000,2,1,289.0000,20,<NA>,NaN,0
2,3,RU14TATN3006,S,103000000,3,1,143.6700,1380,<NA>,NaN,0
3,4,URSI,S,103000000,4,1,1.6200,1100,<NA>,NaN,0
4,5,TGKI,B,103000000,5,1,0.0068,20000000,<NA>,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
883605,883606,SNGS,S,174500000,382041,0,33.8990,104100,<NA>,NaN,9856
883606,883607,EESRP,B,174500000,379207,0,26.0200,600,<NA>,NaN,3450
883607,883608,EESR,S,174500000,375900,0,29.9370,15000,<NA>,NaN,10322
883608,883609,LKOH,S,174500000,377364,0,2039.0000,1,<NA>,NaN,13500


In [5]:
plotno_30_df = OrderLog_df.loc[(OrderLog_df.SECCODE == 'RU0009024277') & (OrderLog_df.TAQPLOTNO == 30)]
plotno_30_df

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,TAQPLOTNO
